In [1]:
!pip install qiskit qiskit-aer --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 51.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 55.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 2.4 MB/s eta 0:00:00


In [ ]:
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
import matplotlib.pyplot as plt

# Initialize the quantum simulator
sim = AerSimulator()

# ============================================================
# Original Circuit for Reference
# ============================================================
print("--- Original Circuit ---")
qc_original = QuantumCircuit(2, 2)
qc_original.h(0)
qc_original.cx(0, 1)
qc_original.cz(0, 1)
qc_original.swap(0, 1)
qc_original.measure_all()

print("Circuit Diagram:")
print(qc_original.draw())

compiled_original = transpile(qc_original, sim)
result_original = sim.run(compiled_original, shots=1024).result()
counts_original = result_original.get_counts()
print("Measurement Counts:", counts_original)
print("-" * 25)

# ============================================================
# Task 1: Vary Control and Target Qubits
# ============================================================
print("\n--- Task 1: Swapping Control and Target Qubits ---")

# We change cx(0, 1) to cx(1, 0) and cz(0, 1) to cz(1, 0)
qc_task1 = QuantumCircuit(2, 2)
qc_task1.h(0)
qc_task1.cx(1, 0)  # Changed: Qubit 1 is now the control
qc_task1.cz(1, 0)  # Changed: Qubit 1 is now the control
qc_task1.swap(0, 1)
qc_task1.measure_all()

print("Circuit Diagram:")
print(qc_task1.draw())

# Run simulation
compiled_task1 = transpile(qc_task1, sim)
result_task1 = sim.run(compiled_task1, shots=1024).result()
counts_task1 = result_task1.get_counts()
print("Measurement Counts:", counts_task1)
print("\nExplanation: By swapping the control and target qubits, the logic of the entanglement changes. The initial Hadamard on qubit 0 has a different effect, leading to a different final state and thus a new distribution of measurement outcomes.")
print("-" * 25)

# ============================================================
# Task 2: Remove the SWAP Gate
# ============================================================
print("\n--- Task 2: Removing the SWAP Gate ---")

qc_task2 = QuantumCircuit(2, 2)
qc_task2.h(0)
qc_task2.cx(0, 1)
qc_task2.cz(0, 1)
# qc_task2.swap(0, 1) # This gate is removed
qc_task2.measure_all()

print("Circuit Diagram:")
print(qc_task2.draw())

# Run simulation
compiled_task2 = transpile(qc_task2, sim)
result_task2 = sim.run(compiled_task2, shots=1024).result()
counts_task2 = result_task2.get_counts()
print("Measurement Counts:", counts_task2)
print("\nExplanation: The SWAP gate physically interchanges the states of the two qubits. By removing it, the measurement applied to qubit 0 now measures the state that would have ended up on qubit 1, and vice-versa. This effectively swaps the labels in the measurement results (e.g., '01' might become '10').")
print("-" * 25)

# ============================================================
# Task 3: Add a Hadamard to the Second Qubit
# ============================================================
print("\n--- Task 3: Adding a Second Hadamard Gate ---")

# Add a Hadamard to qubit 1 before the CNOT gate
qc_task3 = QuantumCircuit(2, 2)
qc_task3.h(0)
qc_task3.h(1)      # Added: Puts qubit 1 into superposition as well
qc_task3.cx(0, 1)
qc_task3.cz(0, 1)
qc_task3.swap(0, 1)
qc_task3.measure_all()

print("Circuit Diagram:")
print(qc_task3.draw())

# Run simulation
compiled_task3 = transpile(qc_task3, sim)
result_task3 = sim.run(compiled_task3, shots=1024).result()
counts_task3 = result_task3.get_counts()
print("Measurement Counts:", counts_task3)
print("\nExplanation: Applying a Hadamard to both qubits creates an equal superposition across all four possible basis states ($|00\rangle, |01\rangle, |10\rangle, |11\rangle$). The subsequent CNOT and CZ gates create a more complex interference pattern, resulting in a significantly different final probability distribution.")
print("-" * 25)


--- 1) Control/Target Variation ---
        ┌───┐┌───┐       ░ ┌─┐   
   q_0: ┤ H ├┤ X ├─■──X──░─┤M├───
        └───┘└─┬─┘ │  │  ░ └╥┘┌─┐
   q_1: ───────■───■──X──░──╫─┤M├
                         ░  ║ └╥┘
   c: 2/════════════════════╬══╬═
                            ║  ║ 
meas: 2/════════════════════╩══╩═
                            0  1 
Measurement Counts: {'10 00': 515, '00 00': 509}
Explanation: Changing control/target qubits changes how entanglement forms. In CNOT and CZ, control determines when the operation happens. Swapping them changes the correlation between qubits.

--- 2) Removing the SWAP gate ---
        ┌───┐         ░ ┌─┐   
   q_0: ┤ H ├──■───■──░─┤M├───
        └───┘┌─┴─┐ │  ░ └╥┘┌─┐
   q_1: ─────┤ X ├─■──░──╫─┤M├
             └───┘    ░  ║ └╥┘
   c: 2/═════════════════╬══╬═
                         ║  ║ 
meas: 2/═════════════════╩══╩═
                         0  1 
Measurement Counts: {'00 00': 538, '11 00': 486}
Explanation: Removing SWAP keeps qubits in their ori